## Functions

__Function for getting the dynamic parameters of the cell model__

In [32]:
def get_dyn_param(data,cell_temp):    
    
    ind = np.where(np.array(data['temps']) == cell_temp)[0][0]

    eta = data['etaParam'][ind]
    Q = data['QParam'][ind]
    gamma = data['GParam'][ind]
    M0 = data['M0Param'][ind]
    M = data['MParam'][ind]
    R0 = data['R0Param'][ind]
    RC = data['RCParam'][ind]
    RC = np.exp(-np.divide(1,RC))
    R = data['RParam'][ind]

    return (eta,Q,gamma,M0,M,R0,RC,R)

__Interpolation Function__

In [33]:
from scipy.interpolate import interp1d

def interp(ocv,SOC):
    
    dum = interp1d(ocv,SOC)
    
    return dum

__OCV calculation for any SOC value Function__

In [34]:
def OCV_from_SOC(z,temp,SOC,OCV_0,OCV_rel):

    # OCV calculation from the given SOC (Coloumb Counting)

    OCV_0_z = interp(SOC,OCV_0)(z)
    OCV_rel_z = interp(SOC,OCV_rel)(z)

    ocv_z = OCV_0_z + temp*OCV_rel_z

    return (ocv_z)

__SOC calculation for any OCV value Function__

In [35]:
def SOC_from_OCV(ocv,temp,OCV,SOC_0,SOC_rel):

    SOC_0_ocv = interp(OCV,SOC_0)(ocv)
    SOC_rel_ocv = interp(OCV,SOC_rel)(ocv)

    soc_ocv = SOC_0_ocv + temp*SOC_rel_ocv
    

    return (soc_ocv)

__OCV and SOC slope__

In [36]:
def docv_from_soc(soc,temp,model):
    
    ocv = OCV_from_SOC(model['SOC'],temp,model['SOC'],model['OCV0'],model['OCVrel'])
    docv = np.diff(ocv)
    dsoc = np.diff(model['SOC'])[0]
    dudz = np.divide(docv,dsoc)
    dOCV = (np.insert(dudz,0,dudz[0]) + np.insert(dudz,-1,dudz[-1]))/2
    dOCVz = interp(model['SOC'],dOCV)(soc)
    
    return dOCVz

__Extended Kalman Filter Algorithm__

In [37]:
def ekf_fn(voltage,current,temp,ekf,model,ir_idx,hk_idx,z_idx):
    
    # Cell Parameters calculated from dynamic model

    eta = get_dyn_param(model,temp)[0]  
    Q = get_dyn_param(model,temp)[1]  
    gamma = get_dyn_param(model,temp)[2]  
    M0 = get_dyn_param(model,temp)[3]  
    M = get_dyn_param(model,temp)[4]  
    R0 = get_dyn_param(model,temp)[5]  
    RC = get_dyn_param(model,temp)[6]  
    R = get_dyn_param(model,temp)[7]
    
    
    vk = voltage
    ik = current

    # If cell is charging
    if ik<0:

        ik = eta*ik

    # Finding the previous model state

    xhat = ekf['xhat']
    sigmax = ekf['sigmax']
    sigmav = ekf['sigmav']
    sigmaw = ekf['sigmaw']
    Qbump = ekf['Qbump']
    I = ekf['prior_I']

    if (np.abs(ik) > Q/100):

        ekf['signik'] = np.sign(ik)

    signik = ekf['signik']

    # ============= EKF matrix calculation ============= 

    nx = len(xhat) + len(RC) -1
    Ahat = np.zeros((nx,nx))
    Bhat = np.zeros((nx,1))
    B = np.zeros((nx,2))

    # SOC EKF matrix
    Ahat[len(RC)+z_idx - 1,len(RC)+z_idx -1] = 1
    Bhat[z_idx + len(RC) -1 ,0] = -dt/(Q*3600)

    # Diffusion current matrix
    Ahat[:len(RC),:len(RC)] = np.diag(RC.flatten())
    Bhat[:len(RC),0] = (1-RC).T

    # Hysterisis matrix
    Ah = np.exp(-np.abs(I*gamma*dt/(3600*Q)))
    Ahat[len(RC)+hk_idx - 1,len(RC)+hk_idx -1] = Ah


    Bhat[len(RC)+hk_idx - 1,0] = (-np.abs(gamma*dt/(Q*3600))*Ah*\
                                  (1+np.sign(I)*xhat[len(RC)+hk_idx - 1]))

    B[len(RC)+hk_idx - 1,1] = Ah-1
    B[:,0] = Bhat.flatten()


    # ============= Step 1a =============
    # State estimate time update

    xhat = np.dot(Ahat,np.array(xhat).reshape(-1,1)) +  np.dot(B,np.array([I,np.sign(I)]).reshape(-1,1))
    xhat[len(RC)+hk_idx - 1] = min(1,max(-1,xhat[len(RC)+hk_idx - 1].item()))
    xhat[z_idx + len(RC) -1] = min(1.05,max(-0.05,xhat[z_idx + len(RC) -1]))

    # ============= Step 1b =============
    # Error covariance update

    sigmax = np.dot(np.dot(Ahat,sigmax),Ahat.T) + np.dot(np.dot(Bhat,sigmaw),Bhat.T)


    # ============= Step 1c =============
    # Output Estimate

    yhat = OCV_from_SOC(xhat[z_idx + len(RC) -1],temp,model['SOC'],model['OCV0'],model['OCVrel'])
    yhat = yhat + M0*signik + M*xhat[len(RC)+hk_idx - 1] - R0*ik - R*xhat[ir_idx]


    # ============= Step 2a =============
    # Estimator gain matrix

    Chat = np.zeros(nx)
    Chat[z_idx] = docv_from_soc(xhat[z_idx],temp,model)
    Chat[hk_idx] = M
    Chat[ir_idx] = -R

    Dhat = np.array(1)

    sigmay = np.dot(np.dot(Chat,sigmax),Chat.T) + np.dot(np.dot(Dhat,sigmav),Dhat.T)

    # Kalman gain
    L = np.dot(sigmax,Chat.T)/sigmay

    # ============= Step 2b =============
    # State estimate measurement update

    r = vk -yhat

    # Checking for sensor errors
    if (r**2 > 100*sigmay):

        L[:] = 0

    xhat = xhat.flatten() + L*r
    xhat[len(RC)+hk_idx - 1] = min(1,max(-1,xhat[len(RC)+hk_idx - 1].item()))
    xhat[z_idx + len(RC) -1] = min(1.05,max(-0.05,xhat[z_idx + len(RC) -1]))

    # ============= Step 2c =============
    # Error covariance measurement update
    sigmax = sigmax - np.dot(np.dot(L,sigmay),L.T)

    # Discarding bad voltage measurement
    if (r**2 > 4*sigmay):

        sigmax[z_idx,z_idx] = sigmax[z_idx,z_idx]*Qbump

    # Performing SVD
    U, S, V = np.linalg.svd(sigmax)

    HH = np.dot(np.dot(V,np.diag(S)),V.T)
    sigmax = (sigmax + sigmax.T + HH + HH.T)/4


    # ============= Update Dictionary =============
    # EKF Model

    ekf['xhat'] = xhat
    ekf['sigmax'] = sigmax
    ekf['prior_I'] = ik

    zk = xhat[z_idx]
    zkbnd = 3*np.sqrt(sigmax[z_idx,z_idx])
    
    return (zk,zkbnd,ekf)

## Main Code

__Model Data__

In [38]:
model = pd.read_pickle('E2model.pkl')

__Measurement Data__

In [39]:
measurement = pd.read_pickle('E2_DYN_15_P05.pkl')

time = [measurement['script3']['time'] - measurement['script3']['time'][0][0]][0][0]
dt = time[1] - time[0]
current = measurement['script3']['current'][0]
voltage = measurement['script3']['voltage'][0]
soc_measured = measurement['script3']['soc'][0] 

__Initial Values for EKF__

In [40]:
ekf = {}

# Temperature
temp = 5 

# Cell Parameters calculated from dynamic model

eta = get_dyn_param(model,temp)[0]  
Q = get_dyn_param(model,temp)[1]  
gamma = get_dyn_param(model,temp)[2]  
M0 = get_dyn_param(model,temp)[3]  
M = get_dyn_param(model,temp)[4]  
R0 = get_dyn_param(model,temp)[5]  
RC = get_dyn_param(model,temp)[6]  
R = get_dyn_param(model,temp)[7]

In [41]:
# Co-variance values

sigmax = np.diag([1e-6,1e-8,2e-4])
sigmav = 2e-1
sigmaw = 2e-1 

# Cell parameters
ir0 = 0
h = 0
soc = SOC_from_OCV(voltage[0],temp,model['OCV'],model['SOC0'],model['SOCrel']) 

# Matrix storage index
ir_idx = 0
hk_idx = 1
z_idx = 2

Qbump = 5
prior_I = 0
signik = 0

In [42]:
# EKF Model

ekf['xhat'] = [ir0,h,soc]
ekf['sigmax'] = sigmax
ekf['sigmav'] = sigmav
ekf['sigmaw'] = sigmaw
ekf['Qbump'] = Qbump
ekf['prior_I'] = prior_I
ekf['signik'] = signik

In [43]:
# Storage variables

sochat = np.zeros(len(soc_measured))
socbound = np.zeros(len(soc_measured))

In [44]:
for i in range(len(voltage)):

    vk = voltage[i]
    ik = current[i]

    [sochat[i],socbound[i],ekf] = ekf_fn(vk,ik,temp,ekf,model,ir_idx,hk_idx,z_idx)


ValueError: A value in x_new is below the interpolation range.

In [ ]:
plt.plot(time[1:]/60,soc_measured[:-1]*100)
plt.plot(time[1:]/60,sochat[:-1]*100)
plt.xlabel('Time (min)')
plt.ylabel('SOC (%)')
plt.legend(['Measured','EKF'])

# plt.plot((sochat-socbound)*100)
# plt.plot((sochat+socbound)*100)

plt.show()

In [ ]:
plt.plot(np.abs(soc_measured - sochat))